In [ ]:
from pathlib import Path
import jsonlines
import pandas as pd
import numpy as np
import json

# metrics_path = Path("metrics") / "wmt19_de_to_en__text-curie-001__200_examples__metrics.json"
metrics_path = Path("metrics") / "wmt19_en_to_de__text-curie-001__200_examples__metrics.jsonl"
score_name = "rougeL_score"

if metrics_path.suffix == ".jsonl":
    with jsonlines.open(metrics_path, 'r') as reader:
        metrics = list(reader.iter())
elif metrics_path.suffix == ".json":
    metrics = json.loads(metrics_path.read_text())
else:
    raise ValueError()

metrics = pd.DataFrame(metrics)
metrics["pred_prob"] = np.exp(metrics["pred_perplexity"])
metrics["echo_prob"] = np.exp(metrics["echo_perplexity"])
# metrics = metrics.loc[metrics["bleu_score"] > 0]

estimators = ["pred_perplexity", "echo_perplexity", "pred_prob", "echo_prob"]
correlation_method = ["pearson","kendall", "spearman"]
correlations = []
for col in estimators:
    for method in correlation_method:
        corr = metrics[col].corr(metrics[score_name], method=method)
        correlations.append({"estimator": col, "method": method, "value": corr})
correlations = pd.DataFrame(correlations)
correlations = correlations.sort_values(["method", "value"], ascending=False)
display(correlations)

for col in estimators:
    metrics.plot(x=col, y=score_name, label=col, kind='scatter')
